In [123]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [124]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [145]:
# skip if result.txt exists
y17 = glob.glob('data/2017/*.txt')
y16 = glob.glob('data/2016/*.txt')
y15 = glob.glob('data/2015/*.txt')
y14 = glob.glob('data/2014/*.txt')
y14_17=y14+y15+y16+y17
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==10:
                    outfile.write(line+'\n')

In [146]:
df=pd.read_table('data/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author','lic']
df.shape

(9123271, 10)

In [147]:
#df=df[df['author']==1]
#df.shape

In [148]:
df['year']=df['year'].astype(int)

In [170]:
df=df[df['year']<=2016][df['year']>=2014]
df.shape

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(9052082, 6)

In [171]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [172]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(9052082, 8)

In [173]:
df=df[['year','count','citation','issnp','issne']].fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [174]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [175]:
s=df.issn.str.split(',',expand=True).stack()

In [176]:
i = s.index.get_level_values(0)

In [177]:
df2 = df.loc[i].copy()

In [178]:
df2["@id"] = s.values

In [179]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation'].div(dg['count']).round(2)
dg.shape

(151837, 5)

In [180]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg_h.citation.apply(lambda x: h_(x))
dg_h.shape

(59135, 3)

In [181]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation'].div(dg_i['count']).round(2)
dg_i.shape

(59135, 4)

In [182]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [183]:
dg_merge=dg_flat.merge(dg_i[['@id','count','citation','impact']])\
.merge(dg_h[['@id','h']]).fillna(0,downcast='infer')
dg_merge.shape

(59135, 14)

In [188]:
batches=dg_merge.shape[0]/10000
for s in range(batches+1):
    dic=[]
    for index, i in dg_merge[10000*s:10000*(s+1)].iterrows():
        journal={}
        journal['@id']=i['@id']
        for y in range(4,7):  
            journal['e:count_'+str(y)]=i['count_'+str(y)]
            journal['e:citation_'+str(y)]=i['citation_'+str(y)]
            journal['e:impact_'+str(y)]=round(i['impact_'+str(y)],2)
        journal['e:count']=i['count']
        journal['e:citation']=i['citation']
        journal['e:impact']=round(i['impact'],2)
        journal['e:h']=i['h']
        if journal not in dic:
            dic.append(journal)
    with open('jsonld/'+str(s)+'journals_stats.jsonld', 'w') as outfile:
        json.dump(dic, outfile)

In [185]:
dg_merge

,@id,count_4,count_5,count_6,citation_4,citation_5,citation_6,impact_4,impact_5,impact_6,count,citation,impact,h
0,0000-992X,1,3,0,0,0,0,0.00,0.00,0.00,4,0,0.00,0
1,0001-0782,298,280,295,1699,696,433,5.70,2.49,1.47,873,2828,3.24,23
2,0001-0898,0,12,0,0,0,0,0.00,0.00,0.00,12,0,0.00,0
3,0001-1452,281,339,360,1533,1388,588,5.46,4.09,1.63,980,3509,3.58,19
4,0001-1541,377,408,463,3405,2018,1015,9.03,4.95,2.19,1248,6438,5.16,27
5,0001-2092,318,328,339,224,184,67,0.70,0.56,0.20,985,475,0.48,7
6,0001-2785,234,233,178,15,7,5,0.06,0.03,0.03,645,27,0.04,2
7,0001-2815,218,199,0,801,471,0,3.67,2.37,0.00,417,1272,3.05,13
8,0001-2998,55,53,58,343,441,209,6.24,8.32,3.60,166,993,5.98,15
9,0001-3072,20,22,33,68,59,34,3.40,2.68,1.03,75,161,2.15,6


In [217]:
tmp=df[df.issnp!='_'][df.issne!='_'][df.issnp<df.issne][['issnp','issne']]


C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [219]:
for row in tmp.iterrows():
    x==1

In [224]:
dg_merge[['@id','count']].to_csv('issn_count.csv',index=False)